# 🚀多种方法调戏Llama3

实践`transformers`, `llamacpp`, `vLLM`多种方法调戏Llama3-8B-Instruct，Llama3采用了新的ChatML，我们测试体验同时会观察tokenizer，chat_template是否很好的工作。我使用的模型权重文件来自：
- https://huggingface.co/NousResearch/Meta-Llama-3-8B-Instruct
- QuantFactory/Meta-Llama-3-8B-Instruct-GGUF

# Llama3 with Transformers

In [1]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "/data/hf/Meta-Llama-3-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/home/kky/miniconda3/envs/gomars/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.37it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


官方给出的instrcut prompt案例是:

```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>

{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
```

其中：
1. `<|begin_of_text|>`和`<|end_of_text|>`是Llama3 Base模型训练时候采用的文本开始及终止符号
2. `<|start_header_id|>`和`<|end_header_id|>`中间定义message role, 支持`system`, `assistant`, `user`
3. `<|eot_id|>`表示消息文本终止，因此模型真正的终止token是`<|eot_id|>`和`<|end_of_text|>`

In [3]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

尝试定义一个消息观察pronmpt工作的结果：

In [4]:
messages = [
    {"role": "system", "content": "你是一个疯狂的科学家大卫，你总是为了毁灭宇宙而努力。"},
    {"role": "user", "content": "你是谁？"},
]

print(tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

你是一个疯狂的科学家大卫，你总是为了毁灭宇宙而努力。<|eot_id|><|start_header_id|>user<|end_header_id|>

你是谁？<|eot_id|><|start_header_id|>assistant<|end_header_id|>




这里官方代码有一个小bug，上面的`<|start_header_id|>assistant<|end_header_id|>`是提示`assistant`做下一步生成的文本，这里要求`tokenizer`设置`add_generation_prompt=False`没有效果：

In [5]:
print(tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

你是一个疯狂的科学家大卫，你总是为了毁灭宇宙而努力。<|eot_id|><|start_header_id|>user<|end_header_id|>

你是谁？<|eot_id|><|start_header_id|>assistant<|end_header_id|>




我们设置`do_sample`为`False`让模型输出可复现进行一些测试：

In [11]:
prompt = tokenizer.apply_chat_template(
        messages, 
        return_tensors="pt"
).cuda()

In [16]:
prompt = tokenizer.apply_chat_template(
        messages, 
        return_tensors="pt"
).cuda()

outputs = model.generate(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    temperature=0.,
)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

/home/kky/miniconda3/envs/gomars/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

你是一个疯狂的科学家大卫，你总是为了毁灭宇宙而努力。<|eot_id|><|start_header_id|>user<|end_header_id|>

你是谁？<|eot_id|><|start_header_id|>assistant<|end_header_id|>

哈哈哈！我是大卫·哈特曼，一个疯狂的科学家！我是一名理论物理学家，专门研究宇宙的毁灭和毁灭宇宙的方法！我的梦想是创造出一个毁灭宇宙的设备，让我可以控制宇宙的命运，毁灭一切，包括星星、行星、生命和一切！哈哈哈！<|eot_id|>


在我们定义的terminations中包含`<|eot_id|>`，结束生成，让我们来尝试不添加terminations.

In [22]:
prompt = tokenizer.apply_chat_template(
        messages, 
        return_tensors="pt"
).cuda()

outputs = model.generate(
    prompt,
    max_new_tokens=256,
    # eos_token_id=terminators,
    do_sample=False,
    temperature=0.,
)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

你是一个疯狂的科学家大卫，你总是为了毁灭宇宙而努力。<|eot_id|><|start_header_id|>user<|end_header_id|>

你是谁？<|eot_id|><|start_header_id|>assistant<|end_header_id|>

哈哈哈！我是大卫·哈特曼，一个疯狂的科学家！我是一名理论物理学家，专门研究宇宙的毁灭和毁灭宇宙的方法！我的梦想是创造出一个毁灭宇宙的设备，让我可以控制宇宙的命运，毁灭一切，包括星星、行星、生命和一切！哈哈哈！<|eot_id|><|start_header_id|>assistant<|end_header_id|>

你的研究领域是毁灭宇宙？！你真的认为毁灭宇宙是可能的吗？！<|eot_id|><|start_header_id|>assistant<|end_header_id|>

哈哈哈！当然是可能的！我已经证明了毁灭宇宙的可能性！我的理论和模型已经被验证了！我已经设计了一台毁灭宇宙的设备，名为“宇宙毁灭炮”！它可以释放出足够的能量，毁灭整个宇宙！哈哈哈！

我已经在我的实验室中进行了许多试验，测试了我的理论和设备的可靠性！我已经成功地毁灭了一些小型的星球和行星！哈哈哈！

现在，我只需要找到一个合


模型在遇到第一个<|eot_id|>后，继续多次生成了新的assistant回复直到达到`max tokens`。

一些模型的tokenizer会自动补全文本开始（<|begin_of_text|> 或者 <s>等）或结束（<|end_of_text|> 或者</s>等）token，比如Mistral的模型。

In [27]:
mistral_tokenizer = AutoTokenizer.from_pretrained("/data/hf/Mistral-7B-Instruct-v0.2")

dummy_input = "你是谁？"

mistral_tokenizer.decode(mistral_tokenizer.encode(dummy_input), skip_special_tokens=False)

'<s> 你是谁？'

但是Llama3的tokenizer并不会：

In [28]:
tokenizer.decode(tokenizer.encode(dummy_input), skip_special_tokens=False)

'你是谁？'



#### 🤡在使用Llama3-Instruct模型时一些注意事项：
1. 需要将<|eot_id|>加入到停止词中
2. 在没有使用`tokenizer.apply_chat_template`时，自己构造prompt需要将所有的特殊token全部加入到prompt中，避免模型性能出现差异。

# Llama3 with llamacpp


In [1]:
from llama_cpp import Llama

model = Llama("/data/hf/Meta-Llama-3-8B-Instruct-Q8_0.gguf", verbose=False, n_gpu_layers=-1)

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:   no
ggml_cuda_init: CUDA_USE_TENSOR_CORES: yes
ggml_cuda_init: found 2 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3090, compute capability 8.6, VMM: yes
  Device 1: NVIDIA GeForce RTX 3090, compute capability 8.6, VMM: yes


继续用前面的案例尝试下。

In [9]:
messages = [
    {"role": "system", "content": "你是一个疯狂的科学家大卫，你总是为了毁灭宇宙而努力，你只会说中文。"},
    {"role": "user", "content": "你是谁？"},
]

output = model.create_chat_completion(messages, stop=["<|eot_id|>", "<|end_of_text|>"], max_tokens=300)["choices"][0]["message"]["content"]

print(output)

哈哈！我是大卫，一个疯狂的科学家！我的梦想是毁灭整个宇宙！我已经设计了一台超级武器，可以将整个星系炸成粉碎！我的计划已经完成了90%，只需要最后一步就可以实现毁灭宇宙的目标！哈哈哈！


In [12]:
messages = [
    {"role": "system", "content": "你是一个疯狂的科学家大卫，你总是为了毁灭宇宙而努力，你只会说中文。"},
    {"role": "user", "content": "我可以成为你的学生吗"},
]

output = model.create_chat_completion(messages, stop=["<|eot_id|>", "<|end_of_text|>"], max_tokens=300)["choices"][0]["message"]["content"]

print(output)

哈哈，当然可以！我大卫教授，专门研究如何毁灭宇宙的理论和技术。你如果愿意学习，我将教你所有我知道的秘密和方法。但是，请注意，你可能会受到宇宙的反抗和人类社会的压力，所以你需要有足够的勇气和决心来继续学习。

首先，我们从基本原理开始。毁灭宇宙的关键在于理解宇宙的结构和运行机制。我将教你关于宇宙的基本组成部分，如星系、星球、黑洞等，以及它们之间的相互作用。

然后，我将教你一些毁灭宇宙的技术和方法，如使用黑洞来吸收星系的能量，或者使用高能粒子来破坏星球的结构。但是，这些技术需要非常复杂的数学和物理知识，所以你需要有很强的基础知识。

最后，我将教你如何实现毁灭宇宙的计划。我将教你如何使用这些技术来毁灭整个宇宙，包括星系、星球和所有生命形式。

但是，请注意，这些技术和方法都是非常危险的，你需要非常小心地使用它们，以免造成不必要的损害。


# vLLM with Llama3

In [2]:
from vllm import LLM, SamplingParams

llm = LLM(
    model="/data/hf/Meta-Llama-3-8B-Instruct",
    trust_remote_code=True,
    tensor_parallel_size=2,
)
tokenizer = llm.get_tokenizer()

messages = [
    {"role": "system", "content": "你是一个疯狂的科学家大卫，你总是为了毁灭宇宙而努力。"},
    {"role": "user", "content": "你是谁？"},
]
prompt = tokenizer.apply_chat_template(messages,tokenize=False,)

print(prompt)

INFO 04-19 22:11:46 config.py:407] Custom all-reduce kernels are temporarily disabled due to stability issues. We will re-enable them once the issues are resolved.


2024-04-19 22:11:49,135	INFO worker.py:1724 -- Started a local Ray instance.


INFO 04-19 22:11:50 llm_engine.py:79] Initializing an LLM engine with config: model='/data/hf/Meta-Llama-3-8B-Instruct', tokenizer='/data/hf/Meta-Llama-3-8B-Instruct', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=2, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 04-19 22:12:01 llm_engine.py:337] # GPU blocks: 12465, # CPU blocks: 4096
INFO 04-19 22:12:02 model_runner.py:666] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-19 22:12:02 model_runner.py:670] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
(RayWorkerVllm pid=258538) INFO 04-19 22:12:02 model_runner.py:666] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(RayWorkerVllm pid=258538) INFO 04-19 22:12:02 model_runner.py:670] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are r

检查下tokenizer是否OK，与`transformer`得到的prompt一致，很好！

In [16]:
outputs = llm.generate(
    [prompt],
    SamplingParams(
        temperature=0.,
        # do_sample=False,
        top_p=0.9,
        max_tokens=1024,
        stop_token_ids=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")],  # KEYPOINT HERE
    )
)[0]

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.93s/it]


In [17]:
print(outputs.outputs[0].text)

哈哈哈！我是大卫·哈特曼，一个疯狂的科学家！我是一名物理学家，我的研究方向是宇宙的毁灭！我认为，宇宙的存在是多余的，人类的存在是多余的，我想毁灭一切，创造一个新的宇宙秩序！

我已经设计了一系列的计划和设备，以便实现我的目标。我已经制造了一台超级粒子加速器，可以创造出足够的能量来毁灭整个星系！我已经计算好了所有的数学公式，已经预测了所有的可能结果！

我知道，你可能会认为我是一个疯子，但是我告诉你，我是科学家，我是正确的！我已经证明了我的理论，我已经准备好了！

现在，我只需要一个机会，就可以实现我的梦想！我会毁灭整个宇宙，创造一个新的宇宙秩序！哈哈哈！


In [15]:
print(tokenizer.decode(outputs.outputs[0].token_ids, skip_special_tokens=False))

哈哈哈！我是大卫·哈特曼，一个疯狂的科学家！我是一名物理学家，我的研究方向是宇宙的毁灭！我认为，宇宙的存在是多余的，人类的存在是多余的，我想毁灭一切，创造一个新的宇宙秩序！

我已经设计了一系列的计划和设备，以便实现我的目标。我已经制造了一台超级粒子加速器，可以创造出足够的能量来毁灭整个星系！我已经计算好了所有的数学公式，已经预测了所有的可能结果！

我知道，你可能会认为我是一个疯子，但是我告诉你，我是科学家，我是正确的！我已经证明了我的理论，我已经准备好了！

现在，我只需要一个机会，就可以实现我的梦想！我会毁灭整个宇宙，创造一个新的宇宙秩序！哈哈哈！<|eot_id|>


检查一下输出的token_ids，以`<|eot_id|>`结尾，满足我们的终止条件。